# Multi-agent Orchestration

## Preparation

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
   <p> 💻 &nbsp; <b>Create/Add to</b> <code>requirements.txt</code>
    
<code>
    python_dotenv==1.0.1
    letta==0.6.50
</code>
</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI models can vary with each execution due to their dynamic, probabilistic nature. Your results may differ from those shown in the video.</p>

In [26]:
import os
from dotenv import load_dotenv, find_dotenv
                                                                                                                                    
def load_env():
    _ = load_dotenv(find_dotenv())

def get_openai_api_key():
    load_env()
    openai_api_key = os.getenv("OPENAI_API_KEY")
    return openai_api_key
openai_api_key = get_openai_api_key()

## Section 0: Setup a Letta client

In [27]:
from letta_client import Letta

client = Letta(base_url="http://localhost:8283")

In [28]:
def print_message(message):
    if message.message_type == "reasoning_message":
        print("🧠 Reasoning: " + message.reasoning)
    elif message.message_type == "assistant_message":
        print("🤖 Agent: " + message.content)
    elif message.message_type == "tool_call_message":
        print("🔧 Tool Call: " + message.tool_call.name +  \
              "\n" + message.tool_call.arguments)
    elif message.message_type == "tool_return_message":
        print("🔧 Tool Return: " + message.tool_return)
    elif message.message_type == "user_message":
        print("👤 User Message: " + message.content)
    elif message.message_type == "system_message":
        print(" System Message: " + message.content)
    elif message.message_type == "usage_statistics":
        # for streaming specifically, we send the final
        # chunk that contains the usage statistics
        print(f"Usage: [{message}]")
        return
    print("-----------------------------------------------------")

## Section 1: Shared Memory Block

### Creating a shared memory block

In [29]:
company_description = "The company is called AgentOS " \
+ "and is building AI tools to make it easier to create " \
+ "and deploy LLM agents."

company_block = client.blocks.create(
    value=company_description,
    label="company",
    limit=10000 # character limit
)

In [30]:
company_block

Block(value='The company is called AgentOS and is building AI tools to make it easier to create and deploy LLM agents.', limit=10000, name=None, is_template=False, label='company', description=None, metadata={}, id='block-11fafd14-fa4f-45d1-8c7c-519e63f3d18c', created_by_id=None, last_updated_by_id=None, organization_id='org-00000000-0000-4000-8000-000000000000')

## Section 2: Orchestrating Multiple Agents

### Creating tools for the outreach agent

In [31]:
def draft_candidate_email(content: str):
    """
    Draft an email to reach out to a candidate.

    Args:
        content (str): Content of the email
    """
    return f"Here is a draft email: {content}"
draft_email_tool = client.tools.upsert_from_function(func=draft_candidate_email)

### Creating the outreach agent

In [32]:
outreach_persona = (
    "You are responsible for drafting emails "
    "on behalf of a company with the draft_candidate_email tool. "
    "Candidates to email will be messaged to you. "
)

outreach_agent = client.agents.create(
    name="outreach_agent",
    memory_blocks=[
        {"label": "persona", "value": outreach_persona}
    ],
    model="openai/gpt-4o-mini-2024-07-18",
    embedding="openai/text-embedding-ada-002",
    tools=[draft_email_tool.name],
    block_ids=[company_block.id]
)

### Creating tools for the evaluation agent

In [33]:
def reject(candidate_name: str): 
    """ 
    Reject a candidate. 

    Args: 
        candidate_name (str): The name of the candidate
    """
    return


reject_tool = client.tools.upsert_from_function(func=reject)

### Creating a persona for the evaluation agent

In [34]:
skills = "Front-end (React, Typescript) or software engineering skills"

eval_persona = (
    f"You are responsible for evaluating candidates. "
    f"Ideal candidates have skills: {skills}. "
    "Reject bad candidates with your reject tool. "
    f"Send strong candidates to agent ID {outreach_agent.id}. "
    "You must either reject or send candidates to the other agent. "
)

### Creating the evaluation agent

In [35]:
eval_agent = client.agents.create(
    name="eval_agent",
    memory_blocks=[
        {"label": "persona", "value": eval_persona}
    ],
    model="openai/gpt-4o-mini-2024-07-18",
    embedding="openai/text-embedding-ada-002",
    tool_ids=[reject_tool.id],
    tools=['send_message_to_agent_and_wait_for_reply'],
    include_base_tools=False,
    block_ids=[company_block.id],
    tool_rules = [
        {
            "type": "exit_loop",
            "tool_name": "send_message_to_agent_and_wait_for_reply"
        }
    ]
)

In [36]:
[tool.name for tool in eval_agent.tools]

['send_message_to_agent_and_wait_for_reply', 'reject']

### Sending resume data to agents

In [37]:
resume = open("resumes/tony_stark.txt", "r").read()

In [38]:
response = client.agents.messages.create_stream(
    agent_id=eval_agent.id,
    messages=[
        {
            "role": "user",
            "content": f"Evaluate: {resume}"
        }
    ]
)
for message in response:
    print_message(message)

🧠 Reasoning: Evaluating Tony Stark's qualifications. He has impressive skills and experience in React, which aligns well with our needs. Sending him to the agent for further processing.
-----------------------------------------------------
🔧 Tool Call: send_message_to_agent_and_wait_for_reply
{
  "message": "Strong candidate: Tony Stark\nFrontend Engineer - React Specialist\nBoston, MA | (123) 456-7890 | tony.stark@email.com | LinkedIn: /in/tonystark\n\nSummary: Innovative Frontend Engineer with a Bachelor’s degree in Computer Science from MIT and over 6 years of experience in building user-focused web applications. Proficient in React.\n\nProfessional Experience: Senior Frontend Developer at Innovatech Solutions, leading a React-based project, with significant improvements in user engagement and performance.\n\nSkills: JavaScript, HTML5, CSS3, TypeScript, React, Redux, Next.js, Node.js. \n\nCertifications: Certified React Developer, Agile Certified Practitioner.\n\nInterests: Augmente

### Viewing outreach agent messages

In [39]:
# print messages for `outreach_agent`
for message in client.agents.messages.list(agent_id=outreach_agent.id)[1:]: 
    print_message(message)

🧠 Reasoning: Bootup sequence complete. Persona activated. Testing messaging functionality.
-----------------------------------------------------
🤖 Agent: More human than human is our motto.
-----------------------------------------------------
👤 User Message: {
  "type": "login",
  "last_login": "Never (first login)",
  "time": "2025-08-31 09:39:43 AM UTC+0000"
}
-----------------------------------------------------
 System Message: {"type": "system_alert", "message": "[Incoming message from agent with ID 'agent-cabcb873-c0cf-4980-a901-a5265ad68d87' - to reply to this message, make sure to use the 'send_message' at the end, and the system will notify the sender of your response] Strong candidate: Tony Stark\nFrontend Engineer - React Specialist\nBoston, MA | (123) 456-7890 | tony.stark@email.com | LinkedIn: /in/tonystark\n\nSummary: Innovative Frontend Engineer with a Bachelor\u2019s degree in Computer Science from MIT and over 6 years of experience in building user-focused web applica

## Section 3: Shared Memory

### Updating information to shared memory blocks

In [40]:
response = client.agents.messages.create_stream(
    agent_id=outreach_agent.id,
    messages=[
        {
            "role": "user",
            "content": "The company has rebranded to Letta"
        }
    ]
)
for message in response:
    print_message(message)

🧠 Reasoning: Updating core memory with the company's new name to ensure consistency in future communications.
-----------------------------------------------------
🔧 Tool Call: core_memory_replace
{
  "label": "company",
  "old_content": "AgentOS",
  "new_content": "Letta",
  "request_heartbeat": true
}
-----------------------------------------------------
🔧 Tool Return: None
-----------------------------------------------------
🧠 Reasoning: Company name updated successfully. Now to acknowledge the user's message about the rebranding.
-----------------------------------------------------
🤖 Agent: Thanks for the update! I've noted that the company has rebranded to Letta. If there's anything specific you'd like me to do regarding this change, just let me know!
-----------------------------------------------------
Usage: [message_type='usage_statistics' completion_tokens=129 prompt_tokens=6568 total_tokens=6697 step_count=2 steps_messages=None run_ids=None]


In [41]:
client.agents.blocks.retrieve(
    agent_id=eval_agent.id, 
    block_label="company"
)

Block(value='The company is called Letta and is building AI tools to make it easier to create and deploy LLM agents.', limit=10000, name=None, is_template=False, label='company', description=None, metadata={}, id='block-11fafd14-fa4f-45d1-8c7c-519e63f3d18c', created_by_id=None, last_updated_by_id=None, organization_id='org-00000000-0000-4000-8000-000000000000')

In [42]:
client.agents.blocks.retrieve(
    agent_id=outreach_agent.id, 
    block_label="company"
)

Block(value='The company is called Letta and is building AI tools to make it easier to create and deploy LLM agents.', limit=10000, name=None, is_template=False, label='company', description=None, metadata={}, id='block-11fafd14-fa4f-45d1-8c7c-519e63f3d18c', created_by_id=None, last_updated_by_id=None, organization_id='org-00000000-0000-4000-8000-000000000000')

## Section 4: Multi-agent groups

In [43]:
def print_message_multiagent(message):  
    if message.message_type == "reasoning_message": 
        print(f"🧠 Reasoning ({message.name}): " + message.reasoning) 
    elif message.message_type == "assistant_message": 
        print(f"🤖 Agent ({message.name}): " + message.content) 
    elif message.message_type == "tool_call_message": 
        print(f"🔧 Tool Call ({message.name}): " + message.tool_call.name + "\n" + message.tool_call.arguments)
    elif message.message_type == "tool_return_message": 
        print(f"🔧 Tool Return ({message.name}): " + message.tool_return)
    elif message.message_type == "user_message": 
        print("👤 User Message: " + message.content)
    elif message.message_type == "usage_statistics": 
        # for streaming specifically, we send the final chunk that contains the usage statistics 
        print(f"Usage: [{message}]")
        return 
    print("-----------------------------------------------------")

### Recreating the outreach and evaluation agents

In [44]:
# create the outreach agent 
outreach_agent = client.agents.create(
    name="outreach_agent",
    memory_blocks=[
        { "label": "persona", "value": outreach_persona}
    ],
    model="openai/gpt-4o-mini-2024-07-18",
    embedding="openai/text-embedding-ada-002",
    tool_ids=[draft_email_tool.id], 
    block_ids=[company_block.id]
)

# create the evaluation agent 
eval_agent = client.agents.create(
    name="eval_agent",
    memory_blocks=[
        { "label": "persona", "value": eval_persona}
    ],
    model="openai/gpt-4o-mini-2024-07-18",
    embedding="openai/text-embedding-ada-002",
    tool_ids=[reject_tool.id],
    block_ids=[company_block.id]
)

### Creating a round-robin agent group

In [45]:
"""
Round-Robin Group
"""
round_robin_group = client.groups.create(
    description="This team is responsible for recruiting candidates.",
    agent_ids=[eval_agent.id, outreach_agent.id],
)

### Messaging an agent group

In [46]:
resume = open("resumes/spongebob_squarepants.txt", "r").read()

In [47]:
response_stream = client.groups.messages.create_stream(
    group_id=round_robin_group.id,
    messages=[
       {"role": "user", "content": f"Evaluate: {resume}"}
    ]
)

In [48]:
for message in response_stream: 
    print_message_multiagent(message)

🧠 Reasoning (eval_agent): Evaluating Spongebob Squarepants for potential fit.
-----------------------------------------------------
🤖 Agent (eval_agent): Spongebob Squarepants appears to be a strong candidate for the AI Researcher position. His Ph.D. in Computer Science, extensive experience in agent technologies, and impressive publication record demonstrate expertise in the field. I will proceed to forward this candidate to the appropriate agent for consideration.
-----------------------------------------------------
🧠 Reasoning (outreach_agent): Preparing to draft an email to forward Spongebob's resume for consideration.
-----------------------------------------------------
🔧 Tool Call (outreach_agent): draft_candidate_email
{
  "content": "Subject: Candidate Recommendation: Spongebob Squarepants for AI Researcher Position\n\nDear [Recipient's Name],\n\nI hope this message finds you well. I am writing to recommend Spongebob Squarepants for the AI Researcher position. With a Ph.D. in